<a href="https://colab.research.google.com/github/abhishekchandar/IN-rainfall-analysis/blob/main/%232_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



 




1.   LR, ,FLAPP, MLP, K-NN, ANN and MANN are usually called data-drivenmodels because they 
capture the mapping between input (e.g.,antecedent rainfall) and output variables (forecasted rainfall) without directly considering the physical laws that underlie the mechanism of rainfall (or precipitation). These models are purely based on the information retrieved from the collected rainfall data.
2.    In order to use statisical model such as ARIMA, non stationary data can be converted into stationary data.

*    MA, PCA and SSA are some options for data preprocessing before giving them as input to a given model

 References:
 https://www.researchgate.net/publication/222522809_Prediction_of_rainfall_time_series_using_modular_artificial_neural_networks_coupled_with_data-preprocessing_techniques

In [153]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn import linear_model

In [154]:
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns

In [155]:
India = pd.read_excel("2.xlsx")

In [156]:
India.head()

,YEAR,ACTL,JUN,JUL,AUG,SEP,JJAS
0,1901,ACTL,110.9,243.2,273.1,124.5,751.7
1,1902,ACTL,109.3,282.5,201.7,200.6,794.0
2,1903,ACTL,116.6,291.7,271.2,200.9,880.5
3,1904,ACTL,165.1,263.1,205.0,131.6,764.8
4,1905,ACTL,92.2,255.0,201.5,179.2,727.9


In [157]:
print('Rows     :',India.shape[0])
print('Columns  :',India.shape[1])
print('\nFeatures :\n     :',India.columns.tolist())
print('\nMissing values    :',India.isnull().values.sum())
print('\nUnique values :  \n',India.nunique())

Rows     : 119
Columns  : 7

Features :
     : ['YEAR', 'ACTL', 'JUN', 'JUL', 'AUG', 'SEP', 'JJAS']

Missing values    : 0

Unique values :  
 YEAR    119
ACTL      1
JUN     108
JUL     117
AUG     116
SEP     117
JJAS    119
dtype: int64


In [158]:
India.describe()

,YEAR,JUN,JUL,AUG,SEP,JJAS
count,119.000000,119.000000,119.000000,119.000000,119.000000,119.000000
mean,1960.000000,167.354622,291.196639,258.340336,173.045378,889.936975
std,34.496377,35.420153,39.797340,34.698565,36.849995,86.550299
min,1901.000000,89.200000,139.200000,192.700000,96.400000,682.000000
25%,1930.500000,143.050000,268.400000,234.200000,145.050000,821.400000
50%,1960.000000,165.100000,293.100000,257.800000,172.600000,897.000000
75%,1989.500000,191.100000,317.350000,286.750000,198.900000,955.600000
max,2019.000000,274.800000,377.500000,336.500000,282.400000,1096.100000


In [159]:
df = India.drop(["YEAR","ACTL"],axis=1)
df

,JUN,JUL,AUG,SEP,JJAS
0,110.9,243.2,273.1,124.5,751.7
1,109.3,282.5,201.7,200.6,794.0
2,116.6,291.7,271.2,200.9,880.5
3,165.1,263.1,205.0,131.6,764.8
4,92.2,255.0,201.5,179.2,727.9
...,...,...,...,...,...
114,188.6,240.7,204.1,131.6,764.9
115,147.0,308.2,239.3,168.6,863.2
116,171.8,290.4,228.2,153.0,843.5
117,155.9,274.3,240.7,132.4,803.4


In [160]:
# fix random seed for reproducibility
np.random.seed(7)

In [161]:
#normalize the dataset
scaler = MinMaxScaler(feature_range=(0,1))
scaled_df_ci = scaler.fit_transform(df)
scaled_df_ci

array([[0.1169181 , 0.43642467, 0.55910987, 0.15107527, 0.16831683],
       [0.10829741, 0.60134285, 0.06258693, 0.56021505, 0.27046607],
       [0.14762931, 0.63994964, 0.54589708, 0.56182796, 0.47935281],
       [0.40894397, 0.51993286, 0.08553547, 0.18924731, 0.1999517 ],
       [0.01616379, 0.48594209, 0.06119611, 0.44516129, 0.11084279],
       [0.47252155, 0.61099454, 0.3852573 , 0.46666667, 0.50953876],
       [0.33836207, 0.3562736 , 0.8178025 , 0.        , 0.24341946],
       [0.20096983, 0.77926983, 0.8310153 , 0.30322581, 0.56628834],
       [0.6325431 , 0.70541334, 0.28511822, 0.35591398, 0.55107462],
       [0.66109914, 0.44943349, 0.66550765, 0.47580645, 0.60251147],
       [0.55818966, 0.10490978, 0.1550765 , 0.46774194, 0.17725187],
       [0.09482759, 0.8036089 , 0.48191933, 0.12204301, 0.32963052],
       [0.67295259, 0.54637012, 0.01251739, 0.06827957, 0.25356194],
       [0.35721983, 0.88334033, 0.34075104, 0.51935484, 0.62303791],
       [0.36260776, 0.38816618, 0.

In [162]:
#split into train and test sets
train_size = int(len(df_ci) * 0.67)
test_size = len(df_ci) - train_size
train,test = scaled_df_ci[0:train_size,:], scaled_df_ci[train_size:len(df_ci),:]
print(len(train),len(test))

79 40


In [163]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back):
		a = dataset[i:(i+look_back)]
		dataX.append(a)
		dataY.append(dataset[i + look_back])
	return np.array(dataX), np.array(dataY)

In [164]:
# reshape into X=t and Y=t+1
look_back = 5
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
display(trainX.shape)
display(trainY.shape)
# # reshape input to be [samples, time steps, features]
# trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
# testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
# trainX
# reshape input to be [samples, time steps, features]
# trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
# n_features = 5
# trainX = trainX.reshape((trainX.shape[0], trainX.shape[1], n_features))
# testX = testX.reshape((testX.shape[0], testX.shape[1], n_features))

(74, 5, 5)

(74, 5)

In [165]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(look_back,5)))
model.add(Dense(5))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=1000, batch_size=4,verbose=1)

Epoch 1/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.2027
Epoch 2/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0721
Epoch 3/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0488
Epoch 4/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0472
Epoch 5/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0470
Epoch 6/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0463
Epoch 7/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0461
Epoch 8/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0456
Epoch 9/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.0465
Epoch 10/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0455
Epoch 11/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0447
Epoch 12/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0450
Epoch 13/1000
19/19 [====

In [166]:
# make predictions
trainPredict = model.predict(trainX,verbose=0)
testPredict = model.predict(testX,verbose=0)

#Reshaping for inverse_transform
# w = np.zeros((73,1))
# trainPredict = np.append(trainPredict,w,axis=1)

# y = np.zeros((73,1))
# trainY = np.reshape(trainY,(-1,1))
# trainY = np.append(trainY,y,axis=1)


In [ ]:
# x = np.zeros((34,1))
# testPredict = np.append(testPredict,x,axis=1)

# z = np.zeros((34,1))
# testY = np.reshape(testY,(-1,1))
# testY = np.append(testY,z,axis=1)

https://stackoverflow.com/questions/49330195/how-to-use-inverse-transform-in-minmaxscaler-for-a-column-in-a-matrix/49330312

http://scipy.github.io/old-wiki/pages/EricsBroadcastingDoc

In [167]:
# invert predictions (trainPredict and testPredict works! The actual value is the first column alone for both!)
trainPredict = scaler.inverse_transform(trainPredict)
testPredict = scaler.inverse_transform(testPredict)
trainY = scaler.inverse_transform(trainY)
testY = scaler.inverse_transform(testY)

# display(trainPredict) #for comparing the predicted and actual array!
# display(trainY)



In [ ]:
#For test dataset
# testPredict = scaler.inverse_transform(testPredict)
# testY = scaler.inverse_transform(testY)
# display(testPredict)
# display(testY)

In [168]:
# # calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[:], trainPredict[:]))
print('Train Score: %.2f RMSE' % (trainScore))


Train Score: 2.14 RMSE


In [169]:
testScore = math.sqrt(mean_squared_error(testY[:], testPredict[:]))
print('Test Score: %.2f RMSE' % (testScore))

Test Score: 60.37 RMSE


In [170]:
display(testY)
display(testPredict)

array([[ 156.2,  291. ,  235.1,  153.5,  835.8],
       [ 184.1,  261. ,  230.1,  125.7,  800.9],
       [ 135.1,  223.8,  246.9,  152.1,  758. ],
       [ 157.9,  377.5,  301.2,  218.4, 1055. ],
       [ 187.2,  302.5,  238.5,  164.3,  892.5],
       [ 191.4,  288.1,  292.8,  197.8,  970.1],
       [ 185.5,  280.9,  262.7,  136.3,  865.3],
       [ 137.3,  267.4,  286.2,  168.2,  859.2],
       [ 170.5,  306.4,  203.4,  210.2,  890.5],
       [ 204.1,  351.3,  285.3,  149.6,  990.3],
       [ 139.3,  322.3,  268.3,  181.1,  911. ],
       [ 186.6,  265.8,  297.8,  148.1,  898.3],
       [ 172.7,  285.2,  263.8,  151.9,  873.6],
       [ 163.9,  284.8,  250.3,  206.5,  905.4],
       [ 173.5,  267. ,  222.2,  188.9,  851.7],
       [ 185.3,  266.7,  225.4,  135.3,  812.8],
       [ 214.7,  280.7,  206.1,  110.6,  812. ],
       [ 171.2,  139.2,  244.2,  139.3,  693.9],
       [ 165.1,  303.3,  246.6,  180.2,  895.1],
       [ 167.4,  242. ,  255.2,  124.1,  788.8],
       [ 146.4,  335

array([[ 119.39949 ,  282.3221  ,  273.52734 ,  128.08925 ,  810.5078  ],
       [ 166.88121 ,  266.5651  ,  257.55002 ,  168.1111  ,  853.13446 ],
       [ 134.25102 ,  260.788   ,  247.05908 ,  113.615326,  757.6886  ],
       [ 181.42351 ,  292.1558  ,  265.7255  ,  139.14189 ,  881.62085 ],
       [ 116.89176 ,  308.3517  ,  254.75346 ,  161.31618 ,  840.8013  ],
       [ 175.74681 ,  282.4673  ,  304.02203 ,  164.45741 ,  928.87054 ],
       [ 168.69336 ,  322.69278 ,  225.56035 ,  124.45976 ,  842.77045 ],
       [ 171.70464 ,  281.5567  ,  250.04358 ,  124.39418 ,  825.0463  ],
       [ 177.17088 ,  281.245   ,  262.14795 ,  157.21796 ,  871.0813  ],
       [ 126.36978 ,  338.11453 ,  223.91037 ,  111.18169 ,  795.20386 ],
       [ 159.11896 ,  277.86966 ,  199.69414 ,  160.54869 ,  796.74036 ],
       [ 213.10953 ,  305.22122 ,  293.3836  ,  177.64197 ,  994.2277  ],
       [ 156.45888 ,  378.57007 ,  373.66434 ,  282.2741  , 1188.7516  ],
       [ 173.69012 ,  348.6361  ,  289

In [178]:
df.tail()
testPredict.shape
testPredict[30:35]

array([[198.15082 , 235.641   , 277.78583 , 111.713554, 819.5216  ],
       [110.248764, 313.16858 , 279.96344 , 132.24487 , 844.7222  ],
       [133.60329 , 320.52856 , 241.17783 ,  84.321846, 780.9001  ],
       [125.49087 , 285.92963 , 251.59467 , 137.80807 , 795.45557 ],
       [183.5961  , 277.08795 , 298.62448 , 195.23741 , 956.67584 ]],
      dtype=float32)

In [185]:
table,row = [],[]
for i in range(30,35):
  row=[]
  for j in range(5):
    row.append(testY[i][j])
    row.append((testPredict[i][j]))
  table.append(row)
dfa = pd.DataFrame(table, columns =['June Actual', 'June Predicted','July Actual', 'July Predicted','August Actual', 'August Predicted','September Actual', 'September Predicted','JJAS Actual', 'JJAS Predicted']) 
dfa

,June Actual,June Predicted,July Actual,July Predicted,August Actual,August Predicted,September Actual,September Predicted,JJAS Actual,JJAS Predicted
0,188.6,198,240.7,235,204.1,277,131.6,111,764.9,819
1,147.0,110,308.2,313,239.3,279,168.6,132,863.2,844
2,171.8,133,290.4,320,228.2,241,153.0,84,843.5,780
3,155.9,125,274.3,285,240.7,251,132.4,137,803.4,795
4,113.4,183,298.1,277,299.7,298,259.9,195,971.0,956


In [186]:
dfa.to_excel("lstm_pred.xlsx")